# Finetune (training) Hands-on

<https://huggingface.co/docs/transformers/ja/training>

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

## Preparing dataset

データセットを読み込む。データセットは [YelpReviewFull](https://huggingface.co/datasets/Yelp/yelp_review_full) でローカルビジネスのレビューとその点数。

In [2]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

トークナイザ(BERTより)を準備

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

D:\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


データセットをあらかじめトークナイズしちゃう

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

実行時間を短くするために、サブセットで学習&評価する

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Train with Pytorch Trainer

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

モデル: Google BERT を読み込む。FP32でGPUへ読み込まれる。

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

for name, param in model.named_parameters():
    print(f"{name:80s} {str(param.shape):30s} {param.dtype} {param.is_cuda}")

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

bert.embeddings.word_embeddings.weight                                           torch.Size([28996, 768])       torch.float32 False
bert.embeddings.position_embeddings.weight                                       torch.Size([512, 768])         torch.float32 False
bert.embeddings.token_type_embeddings.weight                                     torch.Size([2, 768])           torch.float32 False
bert.embeddings.LayerNorm.weight                                                 torch.Size([768])              torch.float32 False
bert.embeddings.LayerNorm.bias                                                   torch.Size([768])              torch.float32 False
bert.encoder.layer.0.attention.self.query.weight                                 torch.Size([768, 768])         torch.float32 False
bert.encoder.layer.0.attention.self.query.bias                                   torch.Size([768])              torch.float32 False
bert.encoder.layer.0.attention.self.key.weight                              

メトリクスの計算関数を定義。メトリクスには [acculacy](https://huggingface.co/spaces/evaluate-metric/accuracy) を利用する。

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print("logits", logits[0:10])
    predictions = np.argmax(logits, axis=-1)
    print("predictions:", predictions[0:10])
    print("references:", labels[0:10])
    return metric.compute(predictions=predictions, references=labels)

Trainerの引数を作成
* 結果を `test_trainer` ディレクトリに確認
* `epoch` ベース(学習の進行がある程度進んだら)で、評価を実行

In [10]:
training_args = TrainingArguments(output_dir="test_trainer/tutorial", eval_strategy="epoch")

学習≒ファインチューニング実行

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
%%time
trainer.train()

D:\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.205479,0.496000
2,No log,1.017869,0.557000
3,No log,1.001495,0.579000


logits [[-1.0838253  -0.26592827  0.4686988   1.2593491   1.1247147 ]
 [-1.4872512  -0.56552845  0.255923    1.2962977   1.5614707 ]
 [ 0.8988643   0.17326835 -0.38753706 -0.605385   -1.4947287 ]
 [-1.4982878  -0.46567714  0.21035378  1.2758518   1.6634251 ]
 [-0.72361964  0.09455957  0.47738528  0.5073498  -0.7683405 ]
 [-1.6476451  -0.5779936   0.1254545   1.2769508   1.8699628 ]
 [ 0.5998183   0.37572598 -0.24949498 -0.5812157  -1.5529152 ]
 [-1.1376874  -0.20103768  0.6748191   1.2054226   0.5281472 ]
 [-1.3817182  -0.29258457  0.52465945  1.2430221   1.034274  ]
 [-0.6258244   0.08374671  0.38156185  0.5189391  -0.6945604 ]]
predictions: [3 4 0 4 3 4 0 3 3 3]
references: [2 4 1 4 3 4 2 3 2 3]
logits [[-2.0979238  -1.0862132  -0.1442049   1.8486749   1.8928374 ]
 [-2.3867989  -0.9981784   0.33805874  2.0501306   1.5348016 ]
 [ 2.408147    0.4579211  -0.91273725 -1.3471279  -1.3872284 ]
 [-2.205455   -1.1257621  -0.23831445  1.9478086   2.0290444 ]
 [-1.5319378   0.37723896  1.23473

TrainOutput(global_step=375, training_loss=1.1273314615885417, metrics={'train_runtime': 110.2596, 'train_samples_per_second': 27.209, 'train_steps_per_second': 3.401, 'total_flos': 789354427392000.0, 'train_loss': 1.1273314615885417, 'epoch': 3.0})